# Reference:
https://github.com/rapidsai/cuml/blob/branch-21.12/notebooks/nearest_neighbors_demo.ipynb

# Nearest Neighbors

Nearest Neighbors enables the query of the k-nearest neighbors from a set of input samples.

The model can take array-like objects, either in host as NumPy arrays or in device (as Numba or cuda_array_interface-compliant), as well as cuDF DataFrames as the input. 

For information on converting your dataset to cuDF format, refer to the cuDF documentation: https://docs.rapids.ai/api/cudf/stable

For additional information on cuML's Nearest Neighbors implementation: https://docs.rapids.ai/api/cuml/stable/api.html#nearest-neighbors

In [1]:
import cudf
import numpy as np

# generate data
from cuml.datasets import make_blobs

# algo
from cuml.neighbors import NearestNeighbors as cuNearestNeighbors
from sklearn.neighbors import NearestNeighbors as skNearestNeighbors

## Define Parameters

In [2]:
n_samples = 2**17
n_features = 40

n_query = 2**13
n_neighbors = 4
random_state = 0

## Generate Data

### GPU

In [3]:
%%time
device_data, _ = make_blobs(n_samples=n_samples,
                            n_features=n_features,
                            centers=5,
                            random_state=random_state)

device_data = cudf.DataFrame(device_data)

CPU times: user 5.35 s, sys: 4.4 s, total: 9.75 s
Wall time: 12.2 s


In [4]:
# Copy dataset from GPU memory to host memory.
# This is done to later compare CPU and GPU results.
host_data = device_data.to_pandas()

## Scikit-learn Model

## Fit

In [5]:
%%time
knn_sk = skNearestNeighbors(algorithm="brute",
                            n_jobs=-1)
knn_sk.fit(host_data)

CPU times: user 14.1 ms, sys: 0 ns, total: 14.1 ms
Wall time: 11.5 ms


NearestNeighbors(algorithm='brute', n_jobs=-1)

In [6]:
%%time
D_sk, I_sk = knn_sk.kneighbors(host_data[:n_query], n_neighbors)

CPU times: user 3min 1s, sys: 1min 11s, total: 4min 13s
Wall time: 1min 11s


## cuML Model

### Fit

In [7]:
%%time
knn_cuml = cuNearestNeighbors()
knn_cuml.fit(device_data)

CPU times: user 77.5 ms, sys: 0 ns, total: 77.5 ms
Wall time: 342 ms


NearestNeighbors()

In [8]:
%%time
D_cuml, I_cuml = knn_cuml.kneighbors(device_data[:n_query], n_neighbors)

CPU times: user 1.31 s, sys: 1.07 s, total: 2.38 s
Wall time: 2.71 s


## Compare Results

cuML currently uses FAISS for exact nearest neighbors search, which limits inputs to single-precision. This results in possible round-off errors when floats of different magnitude are added. As a result, it's very likely that the cuML results will not match Sciklearn's nearest neighbors exactly. You can read more in the [FAISS wiki](https://github.com/facebookresearch/faiss/wiki/FAQ#why-do-i-get-weird-results-with-brute-force-search-on-vectors-with-large-components).

### Distances

In [9]:
D_sk

array([[0.       , 5.1883035, 5.5061746, 5.543916 ],
       [0.       , 4.8640347, 5.0931845, 5.473928 ],
       [0.       , 5.4981475, 5.5115323, 5.556921 ],
       ...,
       [0.       , 4.994953 , 5.0870776, 5.1374645],
       [0.       , 5.2734127, 5.3836575, 5.5896792],
       [0.       , 5.849371 , 5.925003 , 5.9485135]], dtype=float32)

In [10]:
D_cuml

,0,1,2,3
0,0.022097,5.188312,5.506156,5.543903
1,0.000000,4.864032,5.093163,5.473915
2,0.000000,5.498147,5.511557,5.556923
3,0.000000,5.245930,5.404647,5.437747
4,0.000000,5.297670,5.426928,5.508959
...,...,...,...,...
8187,0.015625,5.325123,5.697139,5.783108
8188,0.000000,5.808571,6.097735,6.110044
8189,0.000000,4.994956,5.087083,5.137442
8190,0.022097,5.273421,5.383657,5.589700


In [11]:
passed = np.allclose(D_sk, D_cuml.values, atol=1e-3)
print('compare knn: cuml vs sklearn distances %s'%('equal'if passed else 'NOT equal'))

compare knn: cuml vs sklearn distances NOT equal


### Indices

In [13]:
I_sk

array([[     0, 112423, 111788,   7009],
       [     1, 124010,  71311, 100566],
       [     2,  24889,   5873,  86677],
       ...,
       [  8189,  91066,  59940,   2801],
       [  8190,  65060,  10651,  67369],
       [  8191,  30590,  70671,  72012]])

In [14]:
I_cuml

,0,1,2,3
0,0,112423,111788,7009
1,1,124010,71311,100566
2,2,24889,5873,86677
3,3,14972,99074,76214
4,4,69192,119447,47765
...,...,...,...,...
8187,8187,20857,57059,35841
8188,8188,29850,68804,128722
8189,8189,91066,59940,2801
8190,8190,65060,10651,67369


In [15]:
sk_sorted = np.sort(I_sk, axis=1)
cuml_sorted = np.sort(I_cuml.values, axis=1)

diff = [list(map(lambda x, y: x - y, ii, jj)) for ii, jj in zip(sk_sorted, cuml_sorted)]

passed = (len(diff[diff!=0]) / n_samples) < 1e-9
print('compare knn: cuml vs sklearn indexes %s'%('equal'if passed else 'NOT equal'))

compare knn: cuml vs sklearn indexes NOT equal
